# Capstone Project - The Battle of the Neighborhoods (Week 2)

## Opening a New Coffee Shop in Madrid

*English Adaptation*

**Introduction: Business Problem**

The aim of this project is to find which is the optimal location to open a new Coffe Shop. Specifically, this report will be targeted to stakeholders interested in opening an "Realfood" restaurant in Madrid, Spain.

The new coffe is supposed to have healthy menus for having lunch and breakfast, in order to be the favourite place to have lunch of business travelers that are in the city for working. For this reason, the target group of the restaurant are male and female workers of companies, with good salaries to afford eating out every day, who are really concerned about the quality of the products that they eat. 

Since there are lots of coffees in Madrid I will try to detect locations that are not already crowded with coffees and near big business hotels. I will use data science to find these areas.


**Data Section**

Based on definition of our problem, for this project it is needed the following data:
- The number of menus restaurants that are in the same zone.
- The number of business hotels in the area.

To explore and target recommended locations across different venues I will access data through FourSquare API.

**Methodology**

The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises five stages:
1. Collect the Data about the coffees shops and hotels in Madrid.
2. Explore and understand the data obteined with FourSquare API.
3. Data preparation to find the nearby of the issues.
4. Visualize the results.
5. Obtein conclusions.



**Execution steps**

In [3]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
! pip install geocoder
import geocoder

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         632 KB  conda-forge
    alsa-lib-1.2.3             |       h516909a_0         560 KB  conda-forge
   

In [4]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

distutils: /opt/conda/envs/Python-3.7-main/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/envs/Python-3.7-main/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/envs/Python-3.7-main/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/envs/Python-3.7-main/include/python3.7m
user = False
home = None
root = None
prefix = None


In [56]:
CLIENT_ID = 'SHE0SPGXKRHM01S0E3RUS3023CGYMIGFZZQ2VVTKMALOL25H' # your Foursquare ID
CLIENT_SECRET = 'C0Q3023XMBZSAPTR5XZKRPH4QNOIUQ4CAEOY3HDDYIOFUFSY' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SHE0SPGXKRHM01S0E3RUS3023CGYMIGFZZQ2VVTKMALOL25H
CLIENT_SECRET:C0Q3023XMBZSAPTR5XZKRPH4QNOIUQ4CAEOY3HDDYIOFUFSY


In [69]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "section": "hotel",
    "near": "Madrid",
    "radius": 15000,
    "limit": 500}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [70]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [71]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Madrid', 'Madrid')

In [72]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 40.43137638577648, 'lng': -3.679702745535284},
  'sw': {'lat': 40.4051957924866, 'lng': -3.718611995553454}},
 240)

In [73]:
d["geocode"]

{'what': '',
 'where': 'madrid',
 'center': {'lat': 40.4165, 'lng': -3.70256},
 'displayString': 'Madrid, Spain',
 'cc': 'ES',
 'geometry': {'bounds': {'ne': {'lat': 40.64339099993895,
    'lng': -3.5178330153857993},
   'sw': {'lat': 40.273124093367976, 'lng': -3.8890639517477053}}},
 'slug': 'madrid',
 'longId': '72057594041045671'}

In [74]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [75]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [76]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 100


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '57756e14498e605edcad54fc',
  'name': 'Salmon Gurú',
  'location': {'address': 'C. Echegaray, 21',
   'lat': 40.41486660560401,
   'lng': -3.6995319613728506,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.41486660560401,
     'lng': -3.6995319613728506}],
   'postalCode': '28014',
   'cc': 'ES',
   'city': 'Madrid',
   'state': 'Madrid',
   'country': 'España',
   'formattedAddress': ['C. Echegaray, 21', '28014 Madrid Madrid', 'España']},
  'categories': [{'id': '4bf58dd8d48988d11e941735',
    'name': 'Cocktail Bar',
    'pluralName': 'Cocktail Bars',
    'shortName': 'Cocktail',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/cocktails_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-57756e14498e605edcad54fc-0'}

In [77]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b72841ff964a520db7c2de3',
  'name': 'Plaza de Santa Ana',
  'location': {'address': 'Pl. de Santa Ana',
   'lat': 40.41463076310542,
   'lng': -3.7010325869307574,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.41463076310542,
     'lng': -3.7010325869307574}],
   'postalCode': '28012',
   'cc': 'ES',
   'city': 'Madrid',
   'state': 'Madrid',
   'country': 'España',
   'formattedAddress': ['Pl. de Santa Ana', '28012 Madrid Madrid', 'España']},
  'categories': [{'id': '4bf58dd8d48988d164941735',
    'name': 'Plaza',
    'pluralName': 'Plazas',
    'shortName': 'Plaza',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4b72841ff964a520db7c2de3-1'}

In [78]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes and hotels" % len(df))
df.head()

{'address': 'C. Echegaray, 21', 'lat': 40.41486660560401, 'lng': -3.6995319613728506, 'labeledLatLngs': [{'label': 'display', 'lat': 40.41486660560401, 'lng': -3.6995319613728506}], 'postalCode': '28014', 'cc': 'ES', 'city': 'Madrid', 'state': 'Madrid', 'country': 'España', 'formattedAddress': ['C. Echegaray, 21', '28014 Madrid Madrid', 'España']}
{'address': 'Pl. de Santa Ana', 'lat': 40.41463076310542, 'lng': -3.7010325869307574, 'labeledLatLngs': [{'label': 'display', 'lat': 40.41463076310542, 'lng': -3.7010325869307574}], 'postalCode': '28012', 'cc': 'ES', 'city': 'Madrid', 'state': 'Madrid', 'country': 'España', 'formattedAddress': ['Pl. de Santa Ana', '28012 Madrid Madrid', 'España']}
{'address': 'Pl. Puerta del Sol', 'lat': 40.41679436967101, 'lng': -3.703562021255493, 'labeledLatLngs': [{'label': 'display', 'lat': 40.41679436967101, 'lng': -3.703562021255493}], 'postalCode': '28013', 'cc': 'ES', 'city': 'Madrid', 'state': 'Madrid', 'country': 'España', 'formattedAddress': ['Pl.

AttributeError: 'NoneType' object has no attribute 'items'

                        uid                 name    shortname address  \
0  57756e14498e605edcad54fc          Salmon Gurú     Cocktail           
1  4b72841ff964a520db7c2de3   Plaza de Santa Ana        Plaza           
2  4adcda37f964a5201f3c21e3       Puerta del Sol        Plaza           
3  5815186538faca7f0db08a89           La Primera   Restaurant           
4  4b7eaa51f964a52008f82fe3  La Cabaña Argentina  Argentinian           

  postalcode        lat       lng  
0      28014  40.414867 -3.699532  
1      28012  40.414631 -3.701033  
2      28013  40.416794 -3.703562  
3      28013  40.419183 -3.697794  
4      28014  40.415696 -3.698974  

In [67]:
DaNang_center = d["geocode"]["center"]
DaNang_center

{'lat': 40.4165, 'lng': -3.70256}

In [82]:
from folium import plugins


map_DaNang = folium.Map(location=[40.4165, -3.70256], zoom_start=13)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_DaNang)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_DaNang.add_child(plugins.HeatMap(hm_data))

map_DaNang

**Result**

In the map above we can see Madrid city and the density of coffee shops and hotels in its city center. The analysis shows 100 isues in radius of 15km. The huge density is south of the area, very close to the train station, where we can see a great number of hotels. In the north, there is also a good proportion between hotels and coffee shops.  

Gran Via, the main street of the city shows less density than other areas. However, it is a commercial crowd street and the target group of the project is supposed to avoid this area. In addition, buying or renting a local in that street is very expensive and there are not free locals neither.

The West part, exactly over Retiro's Park shows like the perfect place to set up the new coffee shop. It is an area with low density, with important and business hotels. Furthermore, the area is crow of enterprises so it is perfect ot attract the target group.



**Conclusion**

To sum up, the aim of the project was to achieve the perfect location to set up a new coffee shop in the city center of Madrid. Data science tools were used to get the best optimal place.

The target group of the coffee shop were business travelers, so the location of hotels and currently coffee shops in Madrid were gotten thanks the FourSquare Api application. 

After having all the neccessarily information in a map, we were able to take a conclusion based on evidences. The best area to open the new coffee shop is in the Goya's neighbourhood (over Retiro's Park), where there are companies and business hotels and the currently density of coffees is low.